# Scraping wikipedia for german public research instititions
## TODO: other things than universities 

relevant:
 - https://en.wikipedia.org/wiki/List_of_universities_in_Germany

In [272]:
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sklearn
import plotly.offline as py
from pandas.io.json import json_normalize
import ast
import os
import matplotlib
import matplotlib.pyplot as plt
import http.client, urllib.request, urllib.parse, urllib.error, base64
import os,json
%matplotlib inline
plt.style.use('fivethirtyeight')
matplotlib.rcParams['figure.figsize'] = (15.0, 8.0)

In [273]:
http = urllib3.PoolManager()

In [274]:
url_syn = "http://wikisynonyms.ipeirotis.com/api/"
def get_synonyms_via_wikipedia(synonym):
    encoded = urllib.parse.quote(synonym, safe='')
    response = http.request('GET', url_syn + encoded)
    data = response.data.decode('utf-8')
    print(synonym)
    if data == "":
        return []
    elif response.status == 404:
        print("unable to get synonyms for:" + synonym)
        return []
    else:
        json_data = json.loads(data)
        res = [term["term"] for term in json_data["terms"]]
        return res

In [283]:
list_germ_univ = "https://en.wikipedia.org/wiki/List_of_universities_in_Germany"
response = http.request('GET', list_germ_univ)
soup = BeautifulSoup(response.data.decode('utf-8'), "html5lib")
base = "https://en.wikipedia.org"
unisL = []
for ul in soup.select("#See_also")[0].parent.find_previous_siblings('ul'):
        for li in ul.find_all('li'):
            for a in li.find_all('a'):
                name = a.getText()
                if name != "":
                    res = get_synonyms_via_wikipedia(name) 
                    unisL.append((a.getText(), base + a['href'], res))  

/Users/LeanderK/.virtualenvs/paper_analysis/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



TU Berlin
RWTH Aachen
Chemnitz University of Technology
RWTH Aachen University
Technical University of Berlin
Brandenburg University of Technology
Braunschweig University of Technology
Chemnitz University of Technology
Clausthal University of Technology
Technical University of Dortmund
Darmstadt University of Technology
Dresden University of Technology
Freiberg University of Mining and Technology
Hamburg University of Technology
University of Hannover
Ilmenau University of Technology
Kaiserslautern University of Technology
Karlsruhe Institute of Technology
Technical University of Munich
University of Stuttgart
University of Augsburg
University of Bamberg
University of Bayreuth
Catholic University of Eichstätt-Ingolstadt
University of Erlangen-Nuremberg
University of Freiburg
Zeppelin University
University of Heidelberg
University of Hohenheim
Karlsruhe Institute of Technology
University of Konstanz
University of Mannheim
Bundeswehr University of Munich
University of Munich
Ukrainian Fr

In [297]:
unis = pd.DataFrame(unisL, columns=["insitution", "wikipedia_url", "synonyms"], dtype="str")
unis["synonyms"] = unis[["synonyms"]].astype(str)
unis = unis.drop_duplicates("synonyms")
unis["insitution"] = unis["insitution"].str.lower()

In [298]:
def explode(df, column_to_explode):
    res = (df
           .set_index([x for x in df.columns if x != column_to_explode])[column_to_explode]
           .apply(lambda x: pd.Series(ast.literal_eval(x)))
           .stack()
           .reset_index())
    res = res.rename(columns={
              res.columns[-2]:'exploded_{}_index'.format(column_to_explode),
              res.columns[-1]: '{}'.format(column_to_explode)})
    return res

In [299]:
unis = explode(unis, 'synonyms')
unis["synonyms"] = unis["synonyms"].str.lower()

In [300]:
nipses = []
for filename in os.listdir("../data/processed/"):
    if filename.endswith(".csv"):
        nips = pd.read_csv("../data/processed/" + filename)
        nips['year'] = filename[-8:9]
        nips['year'] = nips['year'].astype("int")
        nipses.append(nips)
nipses = pd.concat(nipses)

In [303]:
nipses[nipses["AfN"].isin(unis["synonyms"])].shape

(543, 18)

In [333]:
sanityCheck = nipses['AfN'].dropna()
sanityCheck = sanityCheck[~sanityCheck.isin(unis["synonyms"])]
sanityCheck = sanityCheck[sanityCheck.str.contains("deutsche")]
sanityCheck

Series([], Name: AfN, dtype: object)

In [320]:
unis.to_csv("../data/processed/german_universities.csv")

TODO: get real data

In [335]:
insitutes = [("max planck society", "max planck society"), ("fraunhofer society", "fraunhofer society")]
insitutes = pd.DataFrame(institutions, columns=["insitution", "synonyms"], dtype="str")
insitutes.to_csv("../data/processed/german_insitutes.csv")